In [1]:
# Dependencies
import pandas as pd
import requests
import gmaps
from pprint import pprint
import numpy as np

from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [2]:
covid_df = pd.read_csv('Resources/COVID19_data.csv')

grouped_df = covid_df.groupby("location").sum()
grouped_df


,total_cases,new_cases,total_deaths,new_deaths
location,,,,
Afghanistan,880185.0,30451.0,17886.0,683.0
Albania,87856.0,2269.0,2791.0,51.0
Algeria,569664.0,12685.0,46382.0,885.0
Andorra,67531.0,855.0,3809.0,52.0
Angola,5779.0,212.0,310.0,10.0
...,...,...,...,...
Vietnam,29073.0,353.0,0.0,0.0
Western Sahara,887.0,25.0,29.0,1.0
Yemen,22884.0,1089.0,5509.0,293.0


In [3]:
grouped_df.reset_index(inplace=True)
grouped_df

,location,total_cases,new_cases,total_deaths,new_deaths
0,Afghanistan,880185.0,30451.0,17886.0,683.0
1,Albania,87856.0,2269.0,2791.0,51.0
2,Algeria,569664.0,12685.0,46382.0,885.0
3,Andorra,67531.0,855.0,3809.0,52.0
4,Angola,5779.0,212.0,310.0,10.0
...,...,...,...,...,...
206,Vietnam,29073.0,353.0,0.0,0.0
207,Western Sahara,887.0,25.0,29.0,1.0
208,Yemen,22884.0,1089.0,5509.0,293.0
209,Zambia,54639.0,1531.0,534.0,21.0


In [4]:
params = {"key": gkey}


for i,row in grouped_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    country = row['location']
    params['address'] = f"{country}"
    try:
        response = requests.get(base_url, params=params).json()    
        grouped_df.loc[i, 'Lat'] = response["results"][0]["geometry"]["location"]["lat"]
        grouped_df.loc[i, 'Lng'] = response["results"][0]["geometry"]["location"]["lng"]

    except:
        print(f"{country} not found. skipping....")
        pass

grouped_df.head(20)

,location,total_cases,new_cases,total_deaths,new_deaths,Lat,Lng
0,Afghanistan,880185.0,30451.0,17886.0,683.0,33.939110,67.709953
1,Albania,87856.0,2269.0,2791.0,51.0,41.153332,20.168331
2,Algeria,569664.0,12685.0,46382.0,885.0,28.033886,1.659626
3,Andorra,67531.0,855.0,3809.0,52.0,42.506285,1.521801
4,Angola,5779.0,212.0,310.0,10.0,-11.202692,17.873887
5,Anguilla,272.0,3.0,0.0,0.0,18.220554,-63.068615
6,Antigua and Barbuda,2231.0,65.0,226.0,3.0,17.060816,-61.796428
7,Argentina,1198603.0,52444.0,36832.0,1167.0,-38.416097,-63.616672
8,Armenia,617257.0,23909.0,9631.0,420.0,40.069099,45.038189
9,Aruba,8629.0,101.0,195.0,3.0,12.521110,-69.968338


In [5]:
locations = grouped_df[["Lat", "Lng"]]

rating = grouped_df["total_cases"]
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
date_grouped = covid_df.groupby('date').sum()
date_grouped.head(20)


,total_cases,new_cases,total_deaths,new_deaths
date,,,,
01-01-20,27.0,0.0,0.0,0.0
01-02-20,11946.0,2120.0,259.0,46.0
01-03-20,87042.0,1819.0,2979.0,58.0
01-04-20,872373.0,73404.0,43130.0,4622.0
01-05-20,3218934.0,84513.0,233257.0,5525.0
01-06-20,6135857.0,107821.0,371882.0,2915.0
02-01-20,27.0,0.0,0.0,0.0
02-02-20,14554.0,2608.0,305.0,46.0
02-03-20,89136.0,2094.0,3046.0,67.0
